# Search

We start by defining our network as an Adjececny List

In [1]:
adjL=[[2],
    [3,4,5],
    [1,5],
    [5,6],
    Int[],
    [5],
    [3,5]
]

7-element Array{Array{Int64,1},1}:
 [2]    
 [3,4,5]
 [1,5]  
 [5,6]  
 Int64[]
 [5]    
 [3,5]  

We need to create a logical function find an admissable arc

In [2]:
function find_addmissable(L,marked_list,tail)
    for head in L[tail]
        if !(head in marked_list)
            return head
        end
    end
    return -1 # Special value to indicate no addmissable arc 
end

find_addmissable (generic function with 1 method)

Next step is to write the search algorithm. 

In [3]:
function graph_search(s,aL,isDepth::Bool=true)
    # Initialization
    n=length(aL) #number of nodes 
    pred=zeros(Int,n)
    marked=Int[s]
    List=Int[s]
    # Now I can stard doing the actual work 
    while !isempty(List)
        i=isDepth? List[end]:List[1] #This is the current node
        j=find_addmissable(aL,marked,i)
        if j>0 # We found an addmissable arc 
            push!(marked,j)
            pred[j]=i
            push!(List,j)
        else
            isDepth? pop!(List):shift!(List)
        end
    end
    pred
end

graph_search (generic function with 2 methods)

In [4]:
graph_search(1,adjL,false) #Breadth

7-element Array{Int64,1}:
 0
 1
 2
 2
 2
 4
 0

In [5]:
graph_search(1,adjL) # Depth

7-element Array{Int64,1}:
 0
 1
 2
 2
 3
 4
 0

In [6]:
graph_search(7,adjL) # Depth

7-element Array{Int64,1}:
 3
 1
 7
 2
 4
 4
 0

Now we try to use and external library to do the same thing

In [7]:
using LightGraphs

In [8]:
function lg_graph_conv(adjL)
    n=length(adjL)
    g=DiGraph(n)
    for i=1:n
        for j in adjL[i]
            add_edge!(g,i,j)
        end
    end
    g
end
g=lg_graph_conv(adjL)

{7, 11} directed graph

Now we do the depth first

In [9]:
dfs_tree(g, 1)

{7, 5} directed graph

In [10]:
bfs_tree(g,1)

{7, 5} directed graph

In [11]:
b=ans

{7, 5} directed graph

In [12]:
for e in edges(b) println(e) end

1=>2
2=>3
2=>4
2=>5
4=>6


In [13]:
using BenchmarkTools

tr1=@benchmark bfs_tree(g,1)

tr2=@benchmark graph_search(1,adjL,false)

mean(tr1.times)/mean(tr2.times)

15.163189648082472

mean(p.times)

In [14]:
function graphGen(num_nodes::Int,min_arcs::Int,max_arcs::Int)
    adjL=Array{Array{Int}}(num_nodes)
    for i=1:num_nodes
        adjL[i]=randperm(num_nodes)[1:rand(min_arcs:max_arcs)]
    end
    adjL
end

graphGen (generic function with 1 method)

In [15]:
aL=graphGen(1000,7,12);

In [16]:
df1=graph_search(1,aL);

In [17]:
count(x->x==0,df1) # Checking if the graph is fully connected 

1

In [18]:
aL_lg=lg_graph_conv(aL)

{1000, 9395} directed graph

In [19]:
tr1=@benchmark bfs_tree(aL_lg,1)

tr2=@benchmark graph_search(1,aL,false)

mean(tr1.times)/mean(tr2.times)

0.6193042155450362

That shows that for larger networks, we have a situation where the LG does better. 
We can now try to optimize our code 

In [38]:
function find_addmissable!(L,marked_list,currentArc,tail)
    for i=currentArc[tail]:length(L[tail]) # Bad way of doing it woud be: for head in L[tail][currentArc[tail]:end]
        head=L[tail][i]
        if !marked_list[head]
            currentArc[tail]=i+1 # We modify the currentArc
            return head
        end
    end
    return -1 # Special value to indicate no addmissable arc 
end

function graph_search(s,aL,isDepth::Bool=true)
    # Initialization
    n=length(aL) #number of nodes 
    pred=zeros(Int,n)
    marked=falses(n) #CHANGED 
    marked[s]=true # ADDED 
    currentArc=ones(Int,n) # ADDED 
    List=Int[s]
    # Now I can stard doing the actual work 
    while !isempty(List)
        i=isDepth? List[end]:List[1] #This is the current node
        j=find_addmissable!(aL,marked,currentArc,i) #Changed 
        if j>0 # We found an addmissable arc 
            marked[j]=true
            pred[j]=i
            push!(List,j)
        else
            isDepth? pop!(List):shift!(List)
        end
    end
    pred
end

graph_search (generic function with 2 methods)

In [43]:
tr1=@benchmark bfs_tree(aL_lg,1)

tr2=@benchmark graph_search(1,aL,false)

mean(tr1.times)/mean(tr2.times)

33.86437053928256

So with our optimized code we are more than 33 times faster!

The below is some sanity check code. 

In [57]:
t_g=bfs_tree(aL_lg,1)
pred_eq=zeros(Int,length(aL))
for e in edges(t_g) 
    pred_eq[dst(e)]=src(e)
end
pred=graph_search(1,aL,false)
println(count(x->x==0,pred))
println(count(x->x==0,pred_eq))

1
1
